# Знакомство с инструментом PyTorch

## План
В этом ноутбуке посмотрим на базовые возможности PyTorch:

0. Подсказки при работе с Jupyter Notebook.
1. Как создать тензор.
2. Операции с тензором.
3. Функции над тензором.
4. Градиенты в PyTorch.
5. Функции потерь.
6. Слои нейросети.
8. PyTorch и видеокарта.

## Почему именно PyTorch?
Этот инструмент стал популярен в мире DL.
Причин много, но из самых интересных стоит выделить три:
1. PyTorch имеет numpy-подобный интерфейс, поэтому на него легко перейти после numpy.
2. PyTorch умеет автоматически считать градиенты всех вычислений, независимо от количества операций.
Можно учить модели произвольного размера.
3. В PyTorch уже реализовано много часто используемых в DL операций и слоев нейросетей.
4. Все вычисления на PyTorch без головной боли можно перенести на GPU и получить прирост x100 в скорости.

## PyTorch: начало

Если вы используете Google Colab или Kaggle Notebooks,
то у вас уже установлен `pytorch`.
На 2024-02-10 они оба используют версии 2.1.

<details>
<summary>**Если используете личный ноутбук или сервер**</summary>

Ваш ноутбук или сервер должны иметь видеокарту, и `pytorch` должен "увидеть" ее.

Сначала устанавливаем пакет `pytorch`.
Лучше всего это делать в виртуальном окружении (можно и в anaconda).
Выполняем в терминале команду:

```bash
pip install torch
```

Затем открываем интерпретатор Python и выполняем:
```python
import torch
torch.cuda.is_avaliable()
# Должно выдать True
```

Если выдало `True`, то `pytorch` увидел вашу видеокарту и может работать с ней.
Если выдало `False` или ошибку, то рекомендуем прочитать [официальную инструкцию](https://pytorch.org/get-started/locally/) по установке - в ней описано, как установить `pytorch` так, чтобы он "видел" видеокарту.
Если же и инструкция не помогла, то советуем работать в Google Colab или Kaggle Notebooks.

</details>

### Подсказки при работе с Jupyter Notebook

In [1]:
import torch

torch.manual_seed(42)

In [2]:
# torch.sq  # Нажмите <Tab>, чтобы увидеть подсказки

In [3]:
# В Jupyter Notebook можно распечатать документацию к классу или функции
# Для этого нужно написать в конце "?"
torch.Tensor?

Init signature: torch.Tensor(self, /, *args, **kwargs)
Docstring:      <no docstring>
File:           ~/.cache/pypoetry/virtualenvs/start-dl-fEQaQ9Q8-py3.10/lib/python3.10/site-packages/torch/__init__.py
Type:           _TensorMeta
Subclasses:     SparseSemiStructuredTensor, Parameter, UninitializedBuffer, MaskedTensor, FakeTensor, FunctionalTensor

In [4]:
# Можно также распечатать весь исходный код, дописав в конец "??"
torch.nn.functional.mse_loss??

Signature:
torch.nn.functional.mse_loss(
    input: torch.Tensor,
    target: torch.Tensor,
    size_average: Optional[bool] = None,
    reduce: Optional[bool] = None,
    reduction: str = 'mean',
) -> torch.Tensor
Source:   
def mse_loss(
    input: Tensor,
    target: Tensor,
    size_average: Optional[bool] = None,
    reduce: Optional[bool] = None,
    reduction: str = "mean",
) -> Tensor:  # noqa: D400,D402
    r"""mse_loss(input, target, size_average=None, reduce=None, reduction='mean') -> Tensor

    Measures the element-wise mean squared error.
    See :class:`~torch.nn.MSELoss` for details.
    """
    if has_torch_function_variadic(input, target):
        return handle_torch_function(
            mse_loss, (input, target), input, target, size_average=size_average, reduce=reduce, reduction=reduction
        )
    if not (target.size() == input.size()):
        warnings.warn(
            f"Using a target size ({target.size()}) that is different to the input size ({input.size()}

### Тензор: великий и ужасный

Напомним: тензор — это всего лишь набор чисел, расфасованных по осям.
О тензоре можно думать как о матрице — вот только если матрица была двумерна, то тензор может иметь три и более размерности.

Тензор с размерностью 1 — это вектор, список чисел.

Тензор с размерностью 2 — это матрица, то есть список списков чисел.

Тензор с размерностью 3 и больше — это тензор, то есть список списков списков (и т.д.) чисел.


![meme](./meme.png)

#### Как создать тензор
Научимся создавать:
1. Тензор с непредсказуемыми данными (самый простой вариант).
2. Тензор из нулей.
3. Тензор, заполненный одним и тем же числом.
4. Тензор со значениями из нормального распределения.

Также познакомимся с in-place операциями и тем, как с ними не запутаться.

In [5]:
"""
Есть много способов создать тензор в torch.
Посмотрим на некоторые из них.
"""

# Самый простой — запросить тензор определенной размерности

t = torch.Tensor(2, 3, 4)
# Будет заполнен произвольными непредсказуемыми данными
t

tensor([[[0.0000e+00, 1.1632e+33, 2.6302e+20, 6.1949e-04],
         [6.4805e-10, 6.3011e-10, 1.6501e-07, 6.7214e-04],
         [6.7739e-10, 4.1020e-08, 6.4097e-10, 1.4580e-19]],

        [[1.1495e+24, 3.0956e-18, 2.9907e+21, 3.2944e-09],
         [1.0073e-11, 1.0665e-08, 6.7377e-10, 6.7003e-10],
         [6.3075e-10, 6.7016e-10, 8.2188e+20, 2.3878e-18]]])

In [6]:
# Можно спросить, какой размер. Помним, что было (2, 3, 4)
t.size()

torch.Size([2, 3, 4])

In [7]:
# Есть .shape — работает аналогично
t.shape

torch.Size([2, 3, 4])

In [8]:
# shape можно удобно сравнивать с tuple — пригодится в тестах
assert t.shape == (2, 3, 4)

In [9]:
# В torch много функций для самых "ходовых" тензоров.
# Например, создать тензор из нулей
# Сделаем матрицу (5, 3), заполненную нулями
torch.zeros((5, 3))

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [10]:
# Тензор (2, 3, 4), все числа равны 1.
# Обратите внимание на точку после 1. — это значит, что тип float
t = torch.ones((2, 3, 4))
t

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [11]:
# А если точнее — float32.
# На лекции мы знакомились с fp16, fp32 — это оно и есть.
t.dtype

torch.float32

In [12]:
# аналогично можно заполнить любыми числами:
2.7 * torch.ones((3, 2, 4))

tensor([[[2.7000, 2.7000, 2.7000, 2.7000],
         [2.7000, 2.7000, 2.7000, 2.7000]],

        [[2.7000, 2.7000, 2.7000, 2.7000],
         [2.7000, 2.7000, 2.7000, 2.7000]],

        [[2.7000, 2.7000, 2.7000, 2.7000],
         [2.7000, 2.7000, 2.7000, 2.7000]]])

Обратите внимание на квадратные скобки.
По ним видно, что тензор как будто состоит из двух матриц 3x4, соединенных вместе.

In [13]:
# Тензор (2, 3, 2, 4), каждый элемент взят из стандартного нормального распределения
torch.randn((2, 3, 2, 4))

tensor([[[[ 1.9269,  1.4873,  0.9007, -2.1055],
          [ 0.6784, -1.2345, -0.0431, -1.6047]],

         [[-0.7521,  1.6487, -0.3925, -1.4036],
          [-0.7279, -0.5594, -0.7688,  0.7624]],

         [[ 1.6423, -0.1596, -0.4974,  0.4396],
          [-0.7581,  1.0783,  0.8008,  1.6806]]],


        [[[ 1.2791,  1.2964,  0.6105,  1.3347],
          [-0.2316,  0.0418, -0.2516,  0.8599]],

         [[-1.3847, -0.8712, -0.2234,  1.7174],
          [ 0.3189, -0.4245,  0.3057, -0.7746]],

         [[-1.5576,  0.9956, -0.8798, -0.6011],
          [-1.2742,  2.1228, -1.2347, -0.4879]]]])

##### in-place операции
Все рассмотренные выше операции создают **новый** тензор.
Но иногда хочется не создавать новый, а менять существующий.

Для этого есть т.н. "in-place" ("на месте") операции — они меняют тот тензор,
над которым применяются, и не создают никаких других тензоров.

In [14]:
# Создадим тензор из единиц
t = torch.ones((2, 3))
t

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [15]:
# И занулим его. Обратите внимание на нижнее подчеркивание.
t.zero_()
t

tensor([[0., 0., 0.],
        [0., 0., 0.]])

В torch все in-place операции строятся как обычные с нижним подчеркиванием (`_`) в конце.

In [16]:
print(t)
t.random_()
print(t)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[12648521.,  3275686.,    84453.],
        [ 5147423.,  1954303., 15271690.]])


In-place операции позволяют сэкономить память, т.к. создаем в два раза меньше тензоров.
У этого есть обратная сторона: если мы передаем тензор в функцию, то функция может
этот тензор "испортить", поменяв его in-place.

Посмотрим на примере:

In [17]:
def random_like(a: torch.Tensor) -> torch.Tensor:
    """Создать случайный тензор того же размера, что и `a`."""
    # перезатираем `a` - это не очень хорошо
    return a.random_(0, 5)


zero_tensor = torch.zeros((2, 3))
print("zero_tensor до функции:")
print(zero_tensor)

random_tensor = random_like(zero_tensor)
print("Рандомный тензор того же размера:")
print(random_tensor)

print("zero_tensor после функции:")
print(zero_tensor)

zero_tensor до функции:
tensor([[0., 0., 0.],
        [0., 0., 0.]])
Рандомный тензор того же размера:
tensor([[1., 4., 0.],
        [4., 3., 3.]])
zero_tensor после функции:
tensor([[1., 4., 0.],
        [4., 3., 3.]])


In [18]:
# без перезатирания
def random_like(a: torch.Tensor) -> torch.Tensor:
    return torch.randint(0, 5, a.shape, dtype=torch.float32)
    # еще можно одной строкой
    # return torch.randint_like(a, 0, 5)
    # у многих функций есть _like аналоги: zero_like, ones_like


zero_tensor = torch.zeros((2, 3))
print("zero_tensor до функции:")
print(zero_tensor)

random_tensor = random_like(zero_tensor)
print("Рандомный тензор того же размера:")
print(random_tensor)

print("zero_tensor после функции:")
print(zero_tensor)

zero_tensor до функции:
tensor([[0., 0., 0.],
        [0., 0., 0.]])
Рандомный тензор того же размера:
tensor([[1., 0., 0.],
        [0., 0., 1.]])
zero_tensor после функции:
tensor([[0., 0., 0.],
        [0., 0., 0.]])


#### Операции с тензором
Тензор создали, что же с ним можно делать дальше?
Много чего. Мы рассмотрим несколько типов операций:
1. Бинарные — как два тензора могут взаимодействовать.
2. Индексирование — как нарезать тензор на куски.
3. Продвинутое создание и индексирование — закрепим знания.

Первая причина популярности PyTorch: numpy-подобный интерфейс, к которому быстро привыкаешь.
Посмотрим, что нам предлагает этот инструмент.
##### Бинарные операции
Тензоры в PyTorch умеют делать те же операции, что и в NumPy: сложение, умножение, возведение в степень и т.д.
Посмотрим на некоторые из них.

In [19]:
# Тензоры одинаковой размерности можно сложить
2 * torch.ones((2, 3)) + 3 * torch.ones((2, 3))

tensor([[5., 5., 5.],
        [5., 5., 5.]])

In [20]:
# Можно умножать поэлементно
a = torch.eye(3)
print(a)
b = torch.randint_like(a, 2, 4)
print(b)
print(a * b)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[3., 2., 3.],
        [3., 3., 3.],
        [2., 3., 2.]])
tensor([[3., 0., 0.],
        [0., 3., 0.],
        [0., 0., 2.]])


In [21]:
# Можно умножить матричным умножением
# Обратите внимание на torch.tensor с маленькой буквы — так можно создавать тензор из списка.
# каждая вложенность списка — это новая размерность тензора.
a = torch.tensor([[3, 5]])
print(a)
print(a.shape)
b = torch.tensor(
    [
        [2, 4],
        [4, 2],
    ]
)
print(b)
"""
         2 | 4
[3, 5] *   |    = [3*2 + 5*4, 3*4 + 5*2]
         4 | 2
"""
# оператор @
print(a @ b)

tensor([[3, 5]])
torch.Size([1, 2])
tensor([[2, 4],
        [4, 2]])
tensor([[26, 22]])


In [22]:
# Оператор @ также делает скалярное умножение векторов.
# Результат тоже будет тензором - нуль-мерным тензором.
# Чтобы превратить его в число, используется .item()
p = torch.tensor([3, 4]) @ torch.tensor([5, 6])
print(p)
print(p.item())

tensor(39)
39


##### Индексирования
Индексирование в PyTorch работает так же, как в NumPy — те же квадратные скобки, те же `:`.
Посмотрим на примеры:

In [23]:
a = torch.tensor(
    [
        [2, 3],
        [4, 5],
        [6, 7],
    ]
)
print(a)
# Взять строку с индексом 1 (нумерация идет с нуля)
print(a[1])
# Взять строку с индексом 1, а в ней - то, что по индексу 0
print(a[1, 0])
# Удобнее думать так:
# - была размерность (3, 2), берем по индексу 1 вдоль первой оси
# - остается размерность (2,), берем по индексу 0 вдоль первой оси
# - остается одно число - это 4

tensor([[2, 3],
        [4, 5],
        [6, 7]])
tensor([4, 5])
tensor(4)


In [24]:
# Можно присваивать через те же [].
# Чтобы сказать "все значения", используем :
a = torch.ones((3, 5, 4, 2))
a[:, 3, :, 1] = 2
print(a[:, 3, :, 1])

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])


In [25]:
# Можно забирать отрезок из тензора.
# Левый конец входит, правый не входит.
a = torch.zeros((3, 5))
print(a)
a[0:2, 2:4] = 2
print(a)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 2., 2., 0.],
        [0., 0., 2., 2., 0.],
        [0., 0., 0., 0., 0.]])


##### Продвинутое создание и индексирование

In [26]:
# Тензор с числами в диапазоне.
# Левая граница входит, правая не входит.
a = torch.arange(2, 9, 2)
print(a)
a = torch.arange(2, 9)
print(a)
a = torch.arange(10, -4, -2)
print(a)

tensor([2, 4, 6, 8])
tensor([2, 3, 4, 5, 6, 7, 8])
tensor([10,  8,  6,  4,  2,  0, -2])


В DL часто нужно состыковать размерности на стыке слоев нейросети.
Для этого приходится добавлять размерность тензору, либо же его "повернуть на бок".

Посмотрим, как это делается.

In [27]:
a = torch.arange(5)
# None в индексировании добавляет ось
# -1 в reshape говорит "сам угадай, сколько по оси элементов"
a[:, None] @ torch.tensor([3, 2]).reshape((1, -1))

tensor([[ 0,  0],
        [ 3,  2],
        [ 6,  4],
        [ 9,  6],
        [12,  8]])

In [28]:
a = torch.arange(5 * 5 * 5).reshape((5, 5, 5))
# При индексировании можно явно указать, какие элементы из какого слоя хотим.
# Учтите, что "дырок" в результате быть не должно
print(a[[0, 1, 4], 2:4, [0, 3, 2]])
# Не сработает, т.к. в последней оси взяли 2 элемента, а в остальных 3
# print(a[[0, 1, 4], 2:4, [0, 1]])

tensor([[ 10,  15],
        [ 38,  43],
        [112, 117]])


In [29]:
from torch.testing import assert_close

# Матрицу можно транспонировать
a = torch.tensor([[1, 2], [3, 4]])
print(a)
print(a.T)
# А если это тензор, то при транспонировании лучше указать две оси
a = torch.arange(27).reshape((3, 3, 3))
print(a)
print(a.transpose(0, 1))

# a.transpose(0, 1) - это то же самое, что
result = torch.zeros_like(a)
for i in range(a.shape[2]):
    result[:, :, i] = a[:, :, i].T
# assert_close проверяет тензоры на равенство.
assert_close(result, a.transpose(0, 1))

tensor([[1, 2],
        [3, 4]])
tensor([[1, 3],
        [2, 4]])
tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])
tensor([[[ 0,  1,  2],
         [ 9, 10, 11],
         [18, 19, 20]],

        [[ 3,  4,  5],
         [12, 13, 14],
         [21, 22, 23]],

        [[ 6,  7,  8],
         [15, 16, 17],
         [24, 25, 26]]])


#### Функции над тензором
Большинство функций, которые есть в NumPy над матрицами,
есть и в PyTorch над тензорами.
Рассмотрим самые популярные из этих функций:
- сложение, умножение, вычитание, деление;
- матричное умножение;
- обращение тензора;

Также разберем нетривиальные моменты.

In [30]:
# Тензоры можно возводить в степень, умножать друг на друга.
# Это будет поэлементно!
# Вообще, большинство арифметических операций в pytorch выполняются поэлементно.
# Это +, -, *, /
a = torch.arange(4).reshape((2, 2))
b = torch.arange(4, 8).reshape((2, 2))
print("Поэлементное умножение:")
print(a * b)
print("То же самое, что '*':")
print(a.mul(b))
print('То же самое, что "+", есть inplace-версия a.add_(b):')
print(a.add(b))
print("Возвести в квадрат поэлементно:")
print(a**2)
print("Обратите внимание: в математической литературе A^2 - это не поэлементно")
print("В матем. литературе обычно под A^2 подразумевают следующее:")
print(a @ a)
print("В частности, обратную матрицу надо считать вот так:")
print(a.float().inverse())  # приводим к float, т.к. inverse работает только с ним
print("Но вот a**(-1) лишь каждый элемент обратит:")
print(a.float() ** (-1))  # pytorch не дает обращать integer элементы тензора

Поэлементное умножение:
tensor([[ 0,  5],
        [12, 21]])
То же самое, что '*':
tensor([[ 0,  5],
        [12, 21]])
То же самое, что "+", есть inplace-версия a.add_(b):
tensor([[ 4,  6],
        [ 8, 10]])
Возвести в квадрат поэлементно:
tensor([[0, 1],
        [4, 9]])
Обратите внимание: в математической литературе A^2 - это не поэлементно
В матем. литературе обычно под A^2 подразумевают следующее:
tensor([[ 2,  3],
        [ 6, 11]])
В частности, обратную матрицу надо считать вот так:
tensor([[-1.5000,  0.5000],
        [ 1.0000,  0.0000]])
Но вот a**(-1) лишь каждый элемент обратит:
tensor([[   inf, 1.0000],
        [0.5000, 0.3333]])


## Градиенты в PyTorch
Вторая причина популярности PyTorch: удобная работа с производными операций.

PyTorch умеет считать градиенты автоматически.
Вы делаете любое вычисление, например:
```python
result = my_matrix ** 2
# затем
result.backward()
```
Более подробная [документация](https://pytorch.org/docs/stable/notes/autograd.html).

In [31]:
# requires_grad=True означает, что мы хотим считать градиент по всем элементам тензора
w = torch.tensor([[1, 1], [2, 2]], dtype=float, requires_grad=True)
x = torch.tensor([[5], [3]], dtype=float)
print(w)
final_answer = w @ x
print(final_answer)
one_scalar = final_answer.sum()
one_scalar.backward()
print(w.grad)
# градиент можно брать только от скаляров, следующая строка не сработает:
# final_answer.backward()

tensor([[1., 1.],
        [2., 2.]], dtype=torch.float64, requires_grad=True)
tensor([[ 8.],
        [16.]], dtype=torch.float64, grad_fn=<MmBackward0>)
tensor([[5., 3.],
        [5., 3.]], dtype=torch.float64)


А теперь вручную считаем:
$$
    \begin{bmatrix}
        w_{11} & w_{12} \\
        w_{21} & w_{22}
    \end{bmatrix}
    \begin{bmatrix}
        x_1 \\
        x_2
    \end{bmatrix}
    = \begin{bmatrix}
        w_{11} x_1 + w_{12} x_2 \\
        w_{21} x_1 + w_{22} x_2
    \end{bmatrix}
    \xrightarrow{\sum}
    w_{11} x_1 + w_{12} x_2 + w_{21} x_1 + w_{22} x_2
$$
отсюда видим, что
$$
\frac{\partial L}{\partial w_{11}} = x_1; \quad 
\frac{\partial L}{\partial w_{12}} = x_2; \quad 
\frac{\partial L}{\partial w_{21}} = x_1; \quad 
\frac{\partial L}{\partial w_{22}} = x_2; \quad 
$$
Смотрим на числа выше и убеждаемся, что градиент был подсчитан верно.

In [32]:
# Градиент не будет работать без requires_grad=True
a = torch.tensor([1.0, 2.0])
try:
    torch.sum(a).backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [33]:
# Чтобы суметь подсчитать градиент,
# pytorch сохраняет все промежуточные результаты.
# Иногда не нужно считать градиент (даже если requires_grad=True)
# В таком случае можно попросить не хранить эти промежуточные результаты.
# Это экономит память.
a = torch.tensor([1.0, 0.2], requires_grad=True)
b = a.sum()
c = b **2
c.backward()
print(b.grad)
print()
print(a.grad)

None

tensor([2.4000, 2.4000])


/tmp/ipykernel_73001/3222167747.py:10: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(b.grad)


In [34]:
with torch.no_grad():
    b = a.sum()
# уже не сработает - градиента не было
try:
    b.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [35]:
# Если тензор участвует в нескольких вычислениях, то вызов .backwards() сложит градиенты
w = torch.tensor([1.0, 2.0], requires_grad=True)
x = torch.tensor([3.0, 4.0])
# Производная loss_1 даст 2w*x = [6, 16]
loss_1 = torch.sum(w**2 * x)
# Производная loss_2 даст 2w + x = [2 + 3, 4 + 4] = [5, 8]
loss_2 = torch.sum(w**2 + w * x + 2)
print(w.grad)
loss_1.backward()
print(w.grad)
loss_2.backward()
# Две производные сложились: [6, 16] + [5, 8] = [11, 24]
print(w.grad)

None
tensor([ 6., 16.])
tensor([11., 24.])


### Функции потерь
В обычном ML было много функций потерь: MSE, MAE, MAPE и так далее.
Они есть и pytorch.

In [36]:
import torch.nn.functional as F

# (5 - 2)^2 = 9
loss = F.mse_loss(torch.tensor([2.0]), torch.tensor([5.0]))
print(loss)
# mean(|2 - 5| + |4 - 0|) = mean(3, 4) = 3.5
loss = F.l1_loss(torch.tensor([2.0, 4.0]), torch.tensor([5.0, 0.0]))
print(loss)
# можно не усреднять, а суммировать
loss = F.l1_loss(torch.tensor([2.0, 4.0]), torch.tensor([5.0, 0.0]), reduction="sum")
print(loss)

# От них можно так же брать градиент!
w = torch.tensor([1.0, 2.0], requires_grad=True)
x = torch.tensor([1.0, 1.0])
y_true = torch.tensor(5.0)
y_pred = w @ x
loss = F.mse_loss(y_pred, y_true)
loss.backward()
print("Градиент (y_true - w @ x)^2:")
print(w.grad)

tensor(9.)
tensor(3.5000)
tensor(7.)
Градиент (y_true - w @ x)^2:
tensor([-4., -4.])


## Слои нейросети
На лекции мы узнали, что нейросети строятся из слоев.
Есть ли слои в pytorch?

Да, они есть — их много готовых.
Это третья причина популярности PyTorch: многие слои из мира Deep Learning уже реализованы и готовы к использованию.

В этом ноутбуке мы рассмотрим полносвязный слой, с остальными будем знакомиться в следующих уроках.

In [37]:
import torch.nn as nn

# Слой вида y = w @ x + b
# Веса инициализируются случайными числами
# bias=False означает "b=0 всегда"
lin_1 = nn.Linear(2, 1)
# Принимает тензор размерности (bs, in_features)
# bs - batch_size, размер батча
# in_features - размерность каждого вектора, в нашем случае 2
y = lin_1(torch.ones((3, 2)))
print(y)

tensor([[-0.7425],
        [-0.7425],
        [-0.7425]], grad_fn=<AddmmBackward0>)


In [38]:
# У линейного слоя есть веса (weight) и смещение (bias), его можно получить
print(lin_1.weight)
print()
print(lin_1.bias)
print()
lin_2 = nn.Linear(2, 1, bias=False)
# bias будет None, если его отключить (см. выше)
print(lin_2.bias)

Parameter containing:
tensor([[-0.5532, -0.4757]], requires_grad=True)

Parameter containing:
tensor([0.2864], requires_grad=True)

None


In [39]:
# Но в лекции говорили, что нужна еще нелинейность.
# Ее тоже можно найти в torch.nn
act = nn.Sigmoid()
print(act(torch.ones((2, 4))))
print()
# соберем все воедино
# nn.Sequential позволяет задать несколько слоев подряд
fc = nn.Sequential(nn.Linear(2, 1, bias=True), nn.Sigmoid())
print(fc)
print()
print(fc(torch.ones((3, 2))))

tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Sigmoid()
)

tensor([[0.3841],
        [0.3841],
        [0.3841]], grad_fn=<SigmoidBackward0>)


In [40]:
# В Sequential можно задать имена слоям через OrderedDict
from collections import OrderedDict

fc = nn.Sequential(
    OrderedDict(
        [
            ("i_am_layer", nn.Linear(2, 1)),
            ("i_am_activation", nn.Sigmoid()),
        ]
    )
)
print(fc)
print()
# слои можно достать по имени (как поле) или по индексу
print(fc[0])
print()
# ради такой возможности и заводят слои через OrderedDict
print(fc.i_am_layer)

Sequential(
  (i_am_layer): Linear(in_features=2, out_features=1, bias=True)
  (i_am_activation): Sigmoid()
)

Linear(in_features=2, out_features=1, bias=True)

Linear(in_features=2, out_features=1, bias=True)


## PyTorch и видеокарта
Четвертая причина популярности PyTorch: удобная работа с видеокартой.
Давайте посмотрим, как это делается.

С чем мы познакомимся:
- операции `.to()`, `.cuda()`, `.cpu()` для ускорения вычислений
- утилита `nvidia-smi` для отслеживания здоровья видеокарты
- демонстрация скорости — насколько же видеокарта быстрее процессора?
- какие бывают проблемы с видеокартой и как их решать на примере `device assert triggered`

In [41]:
a = torch.tensor([1.0, 3.0])
print(a)
# Каждый тензор лежит либо в RAM (она принадлежит CPU), либо в GPU - это можно узнать по .device
print(a.device)
print()
# Тензор легко можно перенести на GPU
a = a.to("cuda")
# Теперь у нас тензор на GPU.
print(a)
print(a.device)
print()
# А теперь - обратно на CPU
a = a.to("cpu")
print(a)
print(a.device)
print()
# Перенести на GPU можно также командой .cuda()
# Если тензор уже на видеокарте, то .cuda() ничего не будет делать
a = a.cuda()
print(a)
print(a.device)
print()
# Аналогично можно перенести на CPU через .cpu()
a = a.cpu()
print(a)
print(a.device)
print()

tensor([1., 3.])
cpu

tensor([1., 3.], device='cuda:0')
cuda:0

tensor([1., 3.])
cpu

tensor([1., 3.], device='cuda:0')
cuda:0

tensor([1., 3.])
cpu



In [42]:
# Главное правило - нельзя перемешивать тензоры на видеокарте и тензоры на CPU.
# Либо все операции на CPU, либо на GPU.
# Можно какие-то промежуточные значения гонять туда-сюда, но это будет медленно.
# Так, код ниже не сработает.
torch.zeros((2, 3), device="cuda") + torch.zeros((2, 3), device="cpu")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [43]:
# GPU не безлимитна. Хочется узнать, сколько ресурсов GPU мы занимаем.
# Для этого есть nvidia-smi
!nvidia-smi

Wed Feb 14 23:33:22 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.07              Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080        On  |   00000000:01:00.0  On |                  N/A |
|  0%   36C    P5             24W /  340W |    1487MiB /  16376MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [44]:
# Видим, что процесс python3.10 что-то занимает. Это как раз наши тензоры.
# У вас будут другие числа - все сильно зависит от модели видеокарты и операционной системы.

В чем же такой плюс от работы на GPU?

Давайте увидим сами. Возьмем большой тензор и начнем его умножать большое число раз.

In [ ]:
# Используем benchmark из pytorch - он сделает "честное" вычисление (без кешей, с прогревом и т.п.)
# Подробнее про benchmark в PyTorch: https://pytorch.org/tutorials/recipes/recipes/benchmark.html
import torch.utils.benchmark as benchmark

num_channels = 4096
many_layers = nn.Sequential(*[nn.Linear(num_channels, num_channels)] * 100)
data = torch.randn((3000, num_channels))
# Прогоним через слой 5 раз и подсчитаем среднее/дисперсию
t = benchmark.Timer(
    stmt="many_layers(data)",
    globals={"many_layers": many_layers, "data": data},
    # заметьте, используем 8 ядер процессора
    num_threads=8,
)
print(t.timeit(5))

many_layers(data)
  11.31 s
  1 measurement, 5 runs , 8 threads


In [ ]:
# Повторим эксперимент, но уже на GPU

many_layers_gpu = many_layers.to("cuda")
data_gpu = data.to("cuda")
t = benchmark.Timer(
    stmt="many_layers_gpu(data_gpu)",
    globals={"many_layers_gpu": many_layers_gpu, "data_gpu": data_gpu},
)
print(t.timeit(5))

many_layers_gpu(data_gpu)
  379.86 ms
  1 measurement, 5 runs , 1 thread


В 30 раз быстрее! И это только игрушечный пример.
В реальных сетях разница может быть еще больше.

В чем же подвох? Почему бы все не учить на видеокарте?
Есть две причины, которые остановят нас:
1. У видеокарты очень мало оперативной памяти по сравнению с сервером.
На сервере вы можете поставить и 512 Гб оперативной памяти, и даже 1 Тб.
Но на видеокарте сейчас можно ставить до 80 Гб (Tesla H800).
2. На видеокарте не очень информативные ошибки. Это особенность программы CUDA,
которую PyTorch использует для вычислений на видеокарте.

Первая проблема фундаментальная, ее решают через распределение нагрузки по нескольким видеокартам.
Это делать сложно: нужно научить несколько GPU взаимодействовать друг с другом, распределять равномерно нагрузку
и синхронизировать работу.
На первых порах вам вряд ли придется столкнуться с такой задачей, поэтому пока оставим эту тему.

Рассмотрим вторую проблему с неинформативными ошибками.

In [45]:
# Подсчитаем Binary Cross Entropy loss с элементами вне {0, 1}
nn.BCELoss()(torch.arange(2, 3).float(), torch.arange(2, 3).float())

RuntimeError: all elements of input should be between 0 and 1

In [46]:
# То же самое, но тензор на GPU
nn.BCELoss()(torch.arange(2, 3).float().cuda(), torch.arange(2, 3).float().cuda())

../aten/src/ATen/native/cuda/Loss.cu:94: operator(): block: [0,0,0], thread: [0,0,0] Assertion `input_val >= zero && input_val <= one` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [52]:
# Более того - после ошибки уже ничего нельзя сделать с видеокартой.
# Придется перезапускать ноутбук :(
torch.randn((2, 3), device='cuda')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Из-за таких проблем на видеокарте не рекомендуется отлаживать модели (об этом говорили в лекции).